In [1]:
import sys
sys.path.append('..')
import tensorflow as tf
from helper import read_audio
import string
from pathlib import Path

In [2]:
# prepare label
labels =  list(
    string.ascii_lowercase  # + string.digits
) + [" ","_"]

In [ ]:
# load tf saved model converted from pytorch 
tf_model = tf.saved_model.load('/content/model')

In [4]:
# prepare tf model
model = tf_model.signatures["serving_default"]

In [5]:
fn = "/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ087-000005.wav"

In [6]:
input,sr = read_audio(fn)
input.shape,sr

((88731,), 16000)

In [14]:
input.shape[0] / sr # around 5 seconds audio

5.5456875

In [15]:
from IPython.display import Audio
Audio(fn)

In [7]:
# read label
Path(fn).with_suffix(".txt").read_text()

'bilakah orang inggeris meminta maaf atas penindasan mereka terhadap orang iraq'

In [8]:
# prepare input for tf model
lala = tf.constant(input[None])
input[None].shape,lala

((1, 88731), <tf.Tensor: shape=(1, 88731), dtype=float32, numpy=
 array([[-4.00983336e-05, -1.10793815e-04, -9.31776085e-05, ...,
         -3.15680372e-04, -1.96074951e-04,  0.00000000e+00]], dtype=float32)>)

In [9]:
# do inference
res = model(lala)

In [10]:
# change from idx to char
pred = [labels[i] for i in tf.math.argmax(res["output_0"][0],-1)]
"".join(pred)

'____bii_lla_kahh  ooranng iin__gerriii s  me_miintta mma__aff  _aaataas   penniinddaasaan  meerree_kka_  _terrhhadaap oooraangg _i__raa_____'

In [11]:
# collapse repeated and remove blank token _
pred2 = ["_"]+pred
"".join([i for i,j in zip(pred,pred2) if i!=j and i!="_"]) # can run in parallel for long text, can use joblib or pqdm

'bilakah orang ingeri s meminta maaf atas penindasan mereka terhadap orang ira'